In [7]:
from ultralytics import SAM

In [8]:
sam_model_path = "/mnt/hdd2/anshul5/imseg-SAM2-UNet/sam2_t.pt"
model = SAM(sam_model_path)

model.info()

Model summary: 422 layers, 38,945,986 parameters, 38,945,986 gradients


(422, 38945986, 38945986, 0.0)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import SAM

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super(DoubleConv, self).__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class SAM2UNetEncoder(nn.Module):
    def __init__(self, sam_model_path):
        super(SAM2UNetEncoder, self).__init__()
        self.sam_model = SAM(sam_model_path)

        self.rfb1 = DoubleConv(3, 64)
        self.rfb2 = DoubleConv(64, 128)
        self.rfb3 = DoubleConv(128, 256)
        self.rfb4 = DoubleConv(256, 512)

    def forward(self, x):
        # Assuming sam_model processes input and provides intermediate features
        x = self.sam_model(x)

        x1 = self.rfb1(x)
        x2 = self.rfb2(x1)
        x3 = self.rfb3(x2)
        x4 = self.rfb4(x3)
        return x1, x2, x3, x4

# Example usage
input_tensor = torch.randn(1, 3, 256, 256)
model = SAM2UNetEncoder("sam2_t.pt")
x1, x2, x3, x4 = model(input_tensor)

print(x1.shape, x2.shape, x3.shape, x4.shape)


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.412174224853516. Dividing input by 255.


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.